In [1]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Let us consider we have 4 documents
Doc1 = ["With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders." ]
Doc2 = ["Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data."]
Doc3 = ["He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems."]
Doc4 = ["But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg."]

In [4]:
full_doc = Doc1+Doc2+Doc3+Doc4
full_doc

['With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
 'Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
 'He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
 'But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.']

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

In [ ]:
def data_clean(text):
    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern,'',' '.join(text))
    tokens = [token.strip() for token in text.split()]
    filtered = [token for token in tokens if token.lower() not in stopword_list]
    filtered = ' '.join(filtered)
    return filtered

In [ ]:
data_clean(nltk.word_tokenize(full_doc[0]))


'Union cabinet approving amendments Motor Vehicles Act 2016 caught drunken driving really deep pockets fine payable court enhanced Rs 10000 firsttime offenders'

In [ ]:
def embeddings(word):
    print(word)
    if word in model.key_to_index:
        return model.get_vector(word)
    else:
        return np.zeros(300)

In [ ]:
# Average vector for each document
out_dict = {}
for sen in full_doc:
    average_vector = (np.mean(np.array([embeddings(x) for x in data_clean(nltk.word_tokenize(sen)).split()]), axis=0))
    d1 = {sen: (average_vector)}
    out_dict.update(d1)

Union
cabinet
approving
amendments
Motor
Vehicles
Act
2016
caught
drunken
driving
really
deep
pockets
fine
payable
court
enhanced
Rs
10000
firsttime
offenders
Natural
language
processing
NLP
area
computer
science
artificial
intelligence
concerned
interactions
computers
human
natural
languages
particular
program
computers
process
analyze
large
amounts
natural
language
data
points
public
transport
good
Mumbai
New
Delhi
good
network
suburban
metro
rail
systems
man
behind
wickets
end
watching
keenly
affirmative
nod
Dhoni
India
captain
Rohit
Sharma
promptly
asked
review
Sure
enough
ball
would
clipped
top
middle
leg


In [ ]:
out_dict

{'With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.': array([ 0.0059648 ,  0.07422152,  0.02200317,  0.05176891, -0.05416038,
        -0.11055964,  0.00973372, -0.01471086,  0.06308608,  0.03867409,
        -0.02027199,  0.00665144, -0.01004583,  0.03567227, -0.1430886 ,
         0.01957009,  0.08405928,  0.01300049,  0.00619533, -0.03672374,
         0.0830411 , -0.01095581, -0.0281979 , -0.01133589,  0.088304  ,
        -0.07389138, -0.0822199 ,  0.04952309,  0.03541704, -0.03258722,
        -0.05574174, -0.10178999,  0.03765314,  0.09196888,  0.02263849,
        -0.10769298,  0.0222057 , -0.05080899, -0.03323087,  0.06027499,
         0.00488836, -0.040003  ,  0.09283066, -0.0387158 , -0.09643277,
        -0.09111439, -0.01537531,  0.04396612,  0.00518799, -0.02616189,
        -0.05735085,  0.0395

In [ ]:
def get_sim(query_embedding, average_vec):
    sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vec))]
    return sim

In [ ]:
# Ranking all docs based on similarity
def rankings(query):
    query_words = (np.mean(np.array([embeddings(x) for x in nltk.word_tokenize(query.lower())], dtype=float), axis=0))
    rank = []
    for k, v in out_dict.items():
        rank.append((k, get_sim(query_words, v)))
    rank = sorted(rank, key=lambda t: t[1], reverse=True)
    print("Ranked documents: ")
    return rank

In [ ]:
rankings("hospital")

hospital
Ranked documents: 


[('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.2356751605739692]),
 ('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [0.18409906475253734]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.15243095468964107]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.138687

In [ ]:
rankings("driving")

driving
Ranked documents: 


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [0.3594728772380067]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.19042557661139026]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.1706653724240128]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.0887230